In [71]:
import base64
import binascii
import struct
import os
import sys
from Crypto.Util.number import bytes_to_long 
from Crypto.Util.number import long_to_bytes
import pandas as pd
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import math

In [72]:
file = os.path.join(sys.path[0], "Collected\\ToSort\\150ms\\2021-10-21_17-50-23.lvx")
toFile = os.path.join(sys.path[0], "Collected\\ToSort\\150ms\\rot-2021-10-21_17-50-23.lvx")
path = os.path.join(sys.path[0], "Collected\\ToSort\\150ms\\")
text_file = open(file, "rb")
for fi in os.listdir(path):
    if file.endswith(".lvx"):
        print(os.path.join(path, fi))

C:\Users\usaru\OneDrive\Desktop\school\Senior design project\Data\Single lidar data\Collected\ToSort\150ms\2021-10-21_17-50-23.lvx
C:\Users\usaru\OneDrive\Desktop\school\Senior design project\Data\Single lidar data\Collected\ToSort\150ms\2021-10-21_17-56-32.lvx
C:\Users\usaru\OneDrive\Desktop\school\Senior design project\Data\Single lidar data\Collected\ToSort\150ms\data
C:\Users\usaru\OneDrive\Desktop\school\Senior design project\Data\Single lidar data\Collected\ToSort\150ms\rot-2021-10-21_17-50-23.lvx
C:\Users\usaru\OneDrive\Desktop\school\Senior design project\Data\Single lidar data\Collected\ToSort\150ms\rot-2021-10-21_17-56-32.lvx
C:\Users\usaru\OneDrive\Desktop\school\Senior design project\Data\Single lidar data\Collected\ToSort\150ms\run1


In [73]:
def is_eof(f):
  s = f.read(1)
  if s != b'':    # restore position
    f.seek(-1, os.SEEK_CUR)
  return s == b''

In [74]:
def Rbyte(numByte,f,s):
    i = 0
    buff = bytearray()
    byte = f.read(1)
    buff.extend(byte)
    temp = ''
    while (byte != b"" and i<numByte-1):
        # Do stuff with byte.
        byte = f.read(1)
        buff += byte
        i+=1
    #print(buff)
    #print(s)
    #print( int.from_bytes(buff[0:1],"little"), int.from_bytes(buff[2:3],"little"))
    if(s == "skp"):
        return buff;
    if(s == 's'):
        return buff.decode("utf-8") 
    if(s == 'a'):
        return buff.decode("ascii") 
    if(s == 'i'): 
        return int.from_bytes(buff,byteorder='little',signed="True")
    if(s == 'ui'): 
        #print(buff, "integer")
        return int.from_bytes(buff,byteorder='little',signed="False")
    if(s == 'i32'): 
        tmp = bytearray([0,0,0,0,0,0,0,0]).extend(buff)
        return int.from_bytes(tmp,"little")
    if(s == 'b'):
        return binascii.b2a_hex(buff)
    if(s == 'f'):
#         print(binascii.b2a_hex(buff))
        return struct.unpack('f', buff)
    if(s == 'l'):
        return int.from_bytes(buff, byteorder='little', signed=False)
    if(s == 'uc'):
        if(len(buff) == 1):
            temp = struct.unpack('B', buff)[0]
        else:
            for i in range(len(val)):
                sct = bytearray(struct.unpack('B', buff))
                temp += sct[0]
        return temp

In [75]:
def Wbyte(val,s,l=8):
    if(s == "skp"):
        return 0;
    if(s == 's'):
        return val.encode("utf-8") 
    if(s == 'a'):
        return val.encode("ascii") 
    if(s == 'i'): 
        return int.to_bytes(val,length=l,byteorder='little',signed="True")
    if(s == 'ui'): 
        return int.to_bytes(val,length=l,byteorder='little',signed="False")
    if(s == 'b'):
        return binascii.b2a_hex(val)
    if(s == 'f'):
        return bytearray(struct.pack('f', val))
    if(s == 'l'):
        data = bytearray(long_to_bytes(val,8))
        data.reverse()
        return data
    if(s == 'uc'):
        if(len(val) == 1):
            temp = bytearray(struct.pack('B', val)[0])
        else:
            for i in range(len(val)):
                sct = bytearray(struct.pack('B', val[i:i+1]))
                temp += sct[0]
        return temp

In [76]:
def removepara(s):
    s = s.replace('(', '')
    s = s.replace(')', '')
    return s

In [77]:
def rotate(pitch, roll, yaw,x,y,z):
    
    cosa = math.cos(yaw)
    sina = math.sin(yaw)
    cosb = math.cos(pitch)
    sinb = math.sin(pitch)
    cosc = math.cos(roll)
    sinc = math.sin(roll)
#     print(x,y,z)
    #print(cosa,sina,cosb,sinb,cosc,sinc)
    xx = cosa*cosb
    xy = cosa*sinb*sinc - sina*cosc
    xz = cosa*sinb*cosc + sina*sinc
    yx = sina*cosb
    yy = sina*sinb*sinc + cosa*cosc
    yz = sina*sinb*cosc - cosa*sinc
    zx = -sinb
    zy = cosb*sinc
    zz = cosb*cosc
#     print(xx,xy,xz)
#     print(yx,yy,yz)
#     print(zx,zy,zz)
#     print(cosa,sina,cosb,sinb,cosc,sinc)
    return [xx*x + xy*y + xz*z, yx*x + yy*y + yz*z,zx*x + zy*y + zz*z]

In [78]:
fps = 1

In [79]:
s = 's'
it = 'i'
ui = 'ui'
b = 'b'
a = 'a'
fl = 'f'
l = 'l'
uc = 'uc'
skp = 'skp'
i32 = 'i32'
ptcd = []

with open(file, "rb") as f:
#     print("Sig",Rbyte(16,f,s))
#     print("A",Rbyte(1,f,uc))
#     print("B",Rbyte(1,f,uc))
#     print("C",Rbyte(1,f,uc))
#     print("D",Rbyte(1,f,uc))

#     print("CODE",Rbyte(4,f,b))
    header = Rbyte(24,f,skp)
#     print("Frame Duration",Rbyte(4,f,l))
    Rbyte(4,f,skp)
    header += Wbyte(50,it,4)
#     deviceCount = Rbyte(1,f,uc)
    header += Rbyte(1,f,skp)
#     print("Device_Count",deviceCount)

#         print("Device " + str(i) + " info:")
#         print("Lidar SN Code: ", Rbyte(16,f,s))
#         print("Hub SN Code: ", Rbyte(16,f,b))
#         print("Device index: ", Rbyte(1,f,uc))
#         print("Device Type: ", Rbyte(1,f,uc))
#         print("Extrinsic Enable",Rbyte(1,f,uc))
    device_info = Rbyte(59-24,f,skp)
    Roll = Rbyte(4,f,fl)[0]
    Pitch = Rbyte(4,f,fl)[0]
    Yaw = Rbyte(4,f,fl)[0]
#     Roll = Rbyte(4,f,skp)
#     Pitch = Rbyte(4,f,skp)
#     Yaw = Rbyte(4,f,skp)
    X = Rbyte(4,f,fl)[0]
    Y = Rbyte(4,f,fl)[0]
    Z = Rbyte(4,f,fl)[0]
    print("X: ", X)
    print("Y: ", Y)
    print("Z: ", Z)
    print(Roll,Pitch,Yaw)
    pos_info = Wbyte(Roll,fl) + Wbyte(Pitch,fl) + Wbyte(Yaw,fl) + Wbyte(X,fl) + Wbyte(Y,fl) + Wbyte(Z,fl)
    print(pos_info)

    Roll =  20/180*math.pi
    Pitch = 0
    Yaw = 0
    print("Roll: ", Roll)
    print("Pitch: ", Pitch)
    print("Yaw: ", Yaw)
    num = 0
    frame = 0
#     while(not is_eof(f)):
    frame+=1
    #print("Current Offset ",curr)
    #print("Next Offset: ", nxt)
    #print("Frame Index: ", Rbyte(8,f,l))
    #Pck = Rbyte(8,f,l)
    #print("Package Count: ", Pck)
    num = 88 
    with open(toFile, 'wb') as file_handler:
        file_handler.write(header)
        file_handler.write(device_info)
        file_handler.write(pos_info)
    with open(toFile, 'ab') as file_handler:
        while(not is_eof(f)):
            curr = Rbyte(8,f,l)
            nxt = Rbyte(8,f,l)
            ix = Rbyte(8,f,l)
            fm = Wbyte(curr,l) + Wbyte(nxt,l) + Wbyte(ix,l)
            num+=24
            file_handler.write(fm)
    #         print(frame//fps)
            while(num < nxt):
                buff = bytes()
        #             print("Device Index ", Rbyte(1,f,uc))
        #             print("Version: ", Rbyte(1,f,uc))
        #             print("Slot ID: ", Rbyte(1,f,uc))
        #             print("Lidar ID ", Rbyte(1,f,uc))
                buff += Rbyte(4,f,skp)
                res = Rbyte(1,f,skp)
                #print("Reserved: ", res)
                if(binascii.b2a_hex(res) != binascii.b2a_hex(bytes.fromhex('00'))):
#                     print("error @",i)
#                     print(nxt)
#                     print(num,nxt)
                    raise Exception("reserved not 00")
                buff += res
        #             print("Status Code : ", Rbyte(4,f,b))
        #             print("TimeStamp Type ", Rbyte(1,f,b))
        #             print("Time Stamp : ", Rbyte(8,f,b))
                buff += Rbyte(13,f,skp)
                buff += Rbyte(1,f,skp)
                #print("Data Type: ", DataType)
                num+=19
                file_handler.write(buff)
                #print(num)
                ptcd = []
                for j in range(100):
                    ar = rotate(Roll,Pitch,Yaw,Rbyte(4,f,it)/1000,Rbyte(4,f,it)/1000,Rbyte(4,f,it)/1000)
        #                 if(j%50 == 0):
        #                     print(ar)
                    for i in range(len(ar)):
                        ar[i] = int(ar[i]*1000)
    #                 print(ar)
                    reflec = Rbyte(1,f,skp)
                    ar.append(reflec)
#                     ptcd.append([Rbyte(4,f,skp),Rbyte(4,f,skp),Rbyte(4,f,skp),Rbyte(1,f,skp)])
                    num+=13
                    ptcd.append(ar)
                for arr in ptcd:
#                     print(len(arr[0]),len(arr[1]),len(arr[2]),len(arr[3]))
                    #print(Wbyte(arr[0],it,4) + Wbyte(arr[1],it,4) + Wbyte(arr[2],it,4) + arr[3])
#                     file_handler.write(arr[0] + arr[1] + arr[2] + arr[3])
                    file_handler.write(Wbyte(arr[0],it,4) + Wbyte(arr[1],it,4) + Wbyte(arr[2],it,4) + arr[3])
    #                 print(arr[0])
    #                 print(int.from_bytes(Wbyte(arr[0],it,4),byteorder='little',signed="True"))
                #print(num,nxt)
    #num+=1
    print(num)

C:\Users\usaru\OneDrive\Desktop\school\Senior design project\Data\Single lidar data\Collected\ToSort\150ms\2021-10-21_17-50-23.lvx
X:  0.0
Y:  0.0
Z:  0.0
0.0 0.0 0.0
bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
Roll:  0.3490658503988659
Pitch:  0
Yaw:  0
39605225
